In [1]:
import pandas as pd
import numpy as np
import csv
import re
import unicodedata
from tqdm import tqdm
from ast import literal_eval

In [52]:
data = pd.read_csv('brand_train.csv')

data = data[['name','brand']]
print('資料總筆數:', len(data))
print('品牌總數:', len(data['brand'].unique()))

資料總筆數: 70046
品牌總數: 4068


# 資料清整

In [3]:
# 全形轉半形
def strQ2B(ustring):
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:                            # 全形空格直接轉換
            inside_code = 32
        elif 65281 <= inside_code <= 65374:                # 全形字元（除空格）根據關係轉化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

In [53]:
# 大寫轉小寫
data['name'] = data['name'].str.lower()

for i in tqdm(range(len(data))):
    try:
        data['name'][i] = strQ2B(data['name'][i])
    except:
        pass
data

100%|██████████████████████████████████| 70046/70046 [00:06<00:00, 11608.71it/s]


,name,brand
0,元山熱水瓶ys-540ap,元山
1,356612@姍拉娜治痘洗面乳150g,姍拉娜
2,台鹽海洋鹼性離子水 600ml,台鹽
3,妮傲絲翠果酸深層保養乳液,NEO-TEC妮傲絲翠
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,舒潔
...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8twgd6622hw,惠而浦 Whirlpool
70042,康乃馨成人紙尿褲l,康乃馨
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur
70044,crest長效鎖白牙膏-輕柔鑽白99g,crest


暫存變數

In [5]:
presave = data.copy(deep=True)

In [6]:
data = data[['name','brand']]

# 清理產品名稱多餘字詞

In [54]:
def clean_mssy(text):
    # 中文量詞
    remove_measure = re.sub(r'''(x|\*)?([0-9]+( +)?(入|支|包|片|瓶|罐|盒|杯|(顆)?袋)+裝|[0-9]+\.[0-9]+包|[0-9]+\.[0-9]+片|[0-9]+\.[0-9]+瓶|[0-9]+\.[0-9]+號|[0-9]+\.[0-9]+罐|[0-9]+\.[0-9]+盒|[0-9]+\.[0-9]+組|[0-9]+\.[0-9]+杯|[0-9]+\.[0-9]+公分|[0-9]+\.[0-9]+公克|[0-9]+\.[0-9]+公斤|[0-9]+\.[0-9]+公升|[0-9]+\.[0-9]+升|[0-9]+\.[0-9]+人份|[0-9]+\.[0-9]+支|[0-9]+\.[0-9]+元|[0-9]+\.[0-9]+支入|[0-9]+\.[0-9]+毫升|[0-9]+\.[0-9]+毫克|[0-9]+\.[0-9]+件|[0-9]+\.[0-9]+日|[0-9]+\.[0-9]+克|[0-9]+\.[0-9]+袋|[0-9]+\.[0-9]+斤|[0-9]+\.[0-9]+磅|[0-9]+\.[0-9]+箱|[0-9]+\.[0-9]+裝|[0-9]+\.[0-9]+顆|[0-9]+\.[0-9]+粒|[0-9]+\.[0-9]+錠|[0-9]+\.[0-9]+桶|[0-9]+\.[0-9]+塊|[0-9]+\.[0-9]+束|[0-9]+\.[0-9]+條|[0-9]+\.[0-9]+吋|[0-9]+\.[0-9]+抽|[0-9]+\.[0-9]+個|[0-9]+\.[0-9]+枚|[0-9]+\.[0-9]+張|[0-9]+\.[0-9]+串|[0-9]+\.[0-9]+捲|[0-9]+\.[0-9]+雙|[0-9]+\.[0-9]+加侖|[0-9]+\.[0-9]+公尺|[0-9]+\.[0-9]+頁|[0-9]+\.[0-9]+歲|[0-9]+\.[0-9]+折|[0-9]+\.[0-9]+款|[0-9]+\.[0-9]+本|[0-9]+\.[0-9]+筒|[0-9]+\.[0-9]+入|下殺[0-9]+\.[0-9]|[0-9]+支入|[0-9]+公克|[0-9]+包|[0-9]+片|[0-9]+瓶|[0-9]+號|[0-9]+罐|[0-9]+盒|[0-9]+組|[0-9]+公分|[0-9]+( )?公斤|[0-9]+入(組|盒)|[0-9]+公升|[0-9]+( )?份入|[0-9]+毫升|[0-9]+毫克|[0-9]+克|[0-9]+袋|[0-9]+( )?磅|[0-9]+箱|[0-9]+裝|[0-9]+顆((袋|盒)裝)?|[0-9]+粒|[0-9]+錠|[0-9]+桶|[0-9]+塊|[0-9]+束|[0-9]+條|[0-9]+吋|[0-9]+抽|[0-9]+個|[0-9]+枚(入)?|[0-9]+張|[0-9]+串|[0-9]+捲|[0-9]+雙|[0-9]+加侖|[0-9]+公尺|[0-9]+頁|[0-9]+本|[0-9]+筒|[0-9]+(\+[0-9])?( +)?入|[0-9]+件(組)?|[0-9]+日|[0-9]+升|[0-9]+人份|[0-9]+碗(入)?|[0-9]+杯|[0-9]+(-[0-9]+)?歲|[0-9]+折|[0-9]+款|[0-9]+支|[0-9]+元|[0-9]+( )?吋|[0-9]+斤|[0-9]+坪|下殺[0-9])''',"",str(text))
    
    # 英文量詞
    remove_egmeasure = re.sub(r'''[0-9]+\.[0-9]+ml|[0-9]+\.[0-9]+l|[0-9]+\.[0-9]+oz|[0-9]+\.[0-9]+%|[0-9]+\.[0-9]+g|[0-9]+\.[0-9]+kg|[0-9]+\.[0-9]+cm|[0-9]+\.[0-9]+mm|[0-9]+\.[0-9]+mg|[0-9]+\.[0-9]+m|[0-9]+\.[0-9]+cc|[0-9]+\.[0-9]+c.c.|[0-9]+\.[0-9]+CC|[0-9]+\.[0-9]+gm|[0-9]+\.[0-9]+pg|$[0-9]+\.[0-9]+|[0-9]+ml|[0-9]+ ml|[0-9]+l|[0-9]+oz|[0-9]+g|[0-9]+ g|[0-9]+kg|([0-9]+\*)?[0-9]+cm|[0-9]+mm|[0-9]+mg|[0-9]+m|[0-9]+cc|[0-9]+c.c.|[0-9]+gm|[0-9]+pg|[0-9]+%|$[0-9]''', "", remove_measure)
    
    # 複合詞
    remove_both = re.sub(r'''^[0-9]{6,15}|[0-9]+@|([0-9]+)?/+(包|裝|盒|kg|組|支|瓶|袋|罐|箱|顆)|\d{4}/\d{1,2}/\d{1,2}|\d{4}/\d{1,2}|\d{4}.\d{1,2}.\d{1,2}|\d{4}.\d{1,2}|長[0-9]+(\.[0-9]+)?x寬[0-9]+(\.[0-9]+)?x高([0-9]+\.[0-9]+)?''', "", remove_egmeasure)
    
    # 行銷詞
    remove_buss = re.sub(r'''買一送一|買+[0-9]+送+[0-9]|[0-9]+送+[0-9]|現貨|蝦皮24h|24h|24 h|代收代付\-|代收代付|任選|特惠組|(保存|最低)(期限|效期至|效期至)|效期\d{4}|效期|期限至|期限|效期至|下單|出貨|官方直營|免運(費)?|型號(:)?\w+''', "", remove_both)
    
    # 特殊符號與自訂
    remove_symbol = re.sub(r'''\d\+\d|[0-9]+\.[0-9]+$|[0-9]+\$|\d{1,3}$|^\s+|\s+$|\*[0-9]+[a-z]+|\~[a-z]+\~|^[a-z]\-|\![a-z+]\-|(x|\*)[0-9]|(x|\*)+[0-9]+\.[0-9]|[!;:"'{}'.,/?\@#<>&※〔〕【】「」『』\[\]《》〈〉«»、∥=→↑。\│±()+\-^~_\*\$|♦▼▶★☆❤㊣╳╱♠⚡️◆✱▲↘✬✪™∮☛●◤◢≡⭐️✴️✨☀◖◗®]''', "", remove_buss) 
    
    return remove_symbol

In [55]:
data['clean_name'] = ''
data['clean_name'] = data.name.apply(clean_mssy)

# data.loc[data['clean_name'] == '']
data

,name,brand,clean_name
0,元山熱水瓶ys-540ap,元山,元山熱水瓶ys540ap
1,356612@姍拉娜治痘洗面乳150g,姍拉娜,姍拉娜治痘洗面乳
2,台鹽海洋鹼性離子水 600ml,台鹽,台鹽海洋鹼性離子水
3,妮傲絲翠果酸深層保養乳液,NEO-TEC妮傲絲翠,妮傲絲翠果酸深層保養乳液
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,舒潔,舒潔棉柔舒適迪士尼抽取式衛生紙
...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8twgd6622hw,惠而浦 Whirlpool,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw
70042,康乃馨成人紙尿褲l,康乃馨,康乃馨成人紙尿褲l
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur,日本芮芙茹零矽靈洗髮露頭皮保養
70044,crest長效鎖白牙膏-輕柔鑽白99g,crest,crest長效鎖白牙膏輕柔鑽白


In [11]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [12]:
check_num = data.copy(deep=True)
check_num = check_num[['name','clean_name']]
check_num['tmp'] = ''
check_num['tmp'] = data.clean_name.apply(has_numbers)
check_num = check_num.loc[check_num['tmp'] == 1].drop('tmp', axis=1).reset_index()
check_num

,index,name,clean_name
0,0,元山熱水瓶ys-540ap,元山熱水瓶ys540ap
1,5,飛利浦全自動美式研磨咖啡機 型號:hd7761,飛利浦全自動美式研磨咖啡機 型號hd7
2,13,光泉1╱2加侖鮮乳,光泉1╱鮮乳
3,19,kissme花漾美姬瞬翹自然捲纖長防水睫毛膏-02玫瑰棕6g,kissme花漾美姬瞬翹自然捲纖長防水睫毛膏02玫瑰棕
4,29,【韓國wonder mama】黃金麥飯石鍋2代-灰金耐磨不沾湯鍋組28cm(含蓋),韓國wonder mama黃金麥飯石鍋2代灰金耐磨不沾湯鍋組含蓋
...,...,...,...
18160,70024,媚比琳超持久霧感液態唇膏305貪玩南瓜,媚比琳超持久霧感液態唇膏305貪玩南瓜
18161,70031,adidas disney 米奇短袖上衣 男 gl2224-a/xs,adidas disney 米奇短袖上衣 男 gl2224axs
18162,70034,asd晶圓不鏽鋼單把湯鍋20cm[ns20a2tw],asd晶圓不鏽鋼單把湯鍋ns20a2tw
18163,70037,紐頓s5成貓雞肉鮭魚5.4kg,紐頓s5成貓雞肉鮭魚


In [14]:
check_num.to_csv('./temp/contain_num_brand.csv')

In [15]:
data = data[['name','brand','clean_name']]
data.to_csv('./temp/cleaned_brand.csv')
data

,name,brand,clean_name
0,元山熱水瓶ys-540ap,元山,元山熱水瓶ys540ap
1,356612@姍拉娜治痘洗面乳150g,姍拉娜,姍拉娜治痘洗面乳
2,台鹽海洋鹼性離子水 600ml,台鹽,台鹽海洋鹼性離子水
3,妮傲絲翠果酸深層保養乳液,NEO-TEC妮傲絲翠,妮傲絲翠果酸深層保養乳液
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,舒潔,舒潔棉柔舒適迪士尼抽取式衛生紙
...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8twgd6622hw,惠而浦 Whirlpool,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw
70042,康乃馨成人紙尿褲l,康乃馨,康乃馨成人紙尿褲l
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur,日本芮芙茹零矽靈洗髮露頭皮保養
70044,crest長效鎖白牙膏-輕柔鑽白99g,crest,crest長效鎖白牙膏輕柔鑽白


## 處理brand標籤

In [383]:
data = pd.read_csv('./temp/cleaned_brand.csv')

In [385]:
data = data[['name', 'brand', 'clean_name']]
data

,name,brand,clean_name
0,元山熱水瓶ys-540ap,元山,元山熱水瓶ys540ap
1,356612@姍拉娜治痘洗面乳150g,姍拉娜,姍拉娜治痘洗面乳
2,台鹽海洋鹼性離子水 600ml,台鹽,台鹽海洋鹼性離子水
3,妮傲絲翠果酸深層保養乳液,NEO-TEC妮傲絲翠,妮傲絲翠果酸深層保養乳液
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,舒潔,舒潔棉柔舒適迪士尼抽取式衛生紙
...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8twgd6622hw,惠而浦 Whirlpool,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw
70042,康乃馨成人紙尿褲l,康乃馨,康乃馨成人紙尿褲l
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur,日本芮芙茹零矽靈洗髮露頭皮保養
70044,crest長效鎖白牙膏-輕柔鑽白99g,crest,crest長效鎖白牙膏輕柔鑽白


In [9]:
# 全形轉半形
def strQ2B(ustring):
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:                            # 全形空格直接轉換
            inside_code = 32
        elif 65281 <= inside_code <= 65374:                # 全形字元（除空格）根據關係轉化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

In [10]:
# 拉丁語系轉換，如:biore
def reverse(x: str):
    return ''.join(c for c in unicodedata.normalize('NFD', x)
                  if unicodedata.category(c) != 'Mn')

In [56]:
# 品牌大寫轉小寫
data['clean_brand'] = data['brand'].str.lower()

for i in tqdm(range(len(data))):
    try:
        data['clean_brand'][i] = reverse(strQ2B(data['clean_brand'][i]))
    except:
        pass

100%|██████████████████████████████████| 70046/70046 [00:06<00:00, 10847.36it/s]


In [57]:
data['brand_ch'] = ''
data['brand_en'] = ''
data['cleaned_brand'] = ''

for i in tqdm(range(len(data))):
    dtxt = data['clean_brand'][i]
    
    if not dtxt.isnumeric():
        # 提出中文
        try:
            # 先提出中英複合詞，e.g. 阿q桶麵, 台灣g霸
            data['brand_ch'][i] = re.search(u"[\u4e00-\u9fa5]+[\u0061-\u007a]+[\u4e00-\u9fa5]+", dtxt).group()
        except AttributeError:
            # 刪除英文字符
            data['brand_ch'][i] = re.sub("( |~|-|·|‧|'|\.|\d+|[0-9]+\.[0-9])?[\u0061-\u007a](~| |-|·|‧|'|\.|!|’|&|/|:)?( |\d+|\+|[0-9]+\.[0-9]|&)?|\u200e", "", dtxt)

        try:
            # 確認架構是否為中+數字+英 或 英+數字+中
            tmp = re.search(u"[\u4e00-\u9fa5]+( )?\d+( )?[\u0061-\u007a]+|[\u0061-\u007a]+( )?\d+( )?[\u4e00-\u9fa5]+", dtxt).group()
            data['brand_en'][i] = re.sub("( |\d+|[0-9]+\.[0-9])?[\u4e00-\u9fa5]+[\u0061-\u007a]+[\u4e00-\u9fa5]+( )?|( )?[\u4e00-\u9fa5]+(~| |-|·|‧)?|\u200e", "", dtxt)
        except AttributeError:
            # 提出英文
            data['brand_en'][i] = re.sub("( |\d+|[0-9]+\.[0-9])?[\u4e00-\u9fa5]+[\u0061-\u007a]+[\u4e00-\u9fa5]+( )?|( |\d+|[0-9]+\.[0-9])?[\u4e00-\u9fa5]+(~| |-|·|‧|±|[0-9]+\.[0-9])?(\d+|\+)?|\u200e", "", dtxt)
        
        data['cleaned_brand'][i] = (data['brand_en'][i]+data['brand_ch'][i]).strip()
        
    else:
        data['cleaned_brand'][i] = dtxt.strip()
        pass

100%|███████████████████████████████████| 70046/70046 [00:20<00:00, 3414.70it/s]


In [58]:
data

,name,brand,clean_name,clean_brand,brand_ch,brand_en,cleaned_brand
0,元山熱水瓶ys-540ap,元山,元山熱水瓶ys540ap,元山,元山,,元山
1,356612@姍拉娜治痘洗面乳150g,姍拉娜,姍拉娜治痘洗面乳,姍拉娜,姍拉娜,,姍拉娜
2,台鹽海洋鹼性離子水 600ml,台鹽,台鹽海洋鹼性離子水,台鹽,台鹽,,台鹽
3,妮傲絲翠果酸深層保養乳液,NEO-TEC妮傲絲翠,妮傲絲翠果酸深層保養乳液,neo-tec妮傲絲翠,妮傲絲翠,neo-tec,neo-tec妮傲絲翠
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,舒潔,舒潔棉柔舒適迪士尼抽取式衛生紙,舒潔,舒潔,,舒潔
...,...,...,...,...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8twgd6622hw,惠而浦 Whirlpool,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw,惠而浦 whirlpool,惠而浦,whirlpool,whirlpool惠而浦
70042,康乃馨成人紙尿褲l,康乃馨,康乃馨成人紙尿褲l,康乃馨,康乃馨,,康乃馨
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur,日本芮芙茹零矽靈洗髮露頭皮保養,芮芙茹 reveur,芮芙茹,reveur,reveur芮芙茹
70044,crest長效鎖白牙膏-輕柔鑽白99g,crest,crest長效鎖白牙膏輕柔鑽白,crest,,crest,crest


In [392]:
print('單一品牌總數:', len(data['cleaned_brand'].unique()))
print('英文切割總數:', len(data['brand_ch'].unique()))
print('中文切割總數:', len(data['brand_en'].unique()))

單一品牌總數: 3865
英文切割總數: 2385
中文切割總數: 2377


In [393]:
data.to_csv('./temp/forfinaldf.csv')

## 處理標籤

In [60]:
simpbrand = pd.read_csv('單一品牌內容.csv', converters={"similar": literal_eval})
simpbrand = simpbrand[['brand','similar']]

In [61]:
simpbrand['same'] = ''
tmplen = len(simpbrand)
for i in range(tmplen):
    for name in simpbrand['similar'][i]:
        if simpbrand['brand'][i] != name:
            simpbrand = simpbrand.append({
                "brand": name,
                "similar": [],
                "same": str(simpbrand['brand'][i])
            }, ignore_index=True)

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

...

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/folders/xc/lbpw82tj4pd9lw15l1swq9040000gn/T/ipykernel_68804/3654565440.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simpbrand = simpbrand.append({
/var/fol

In [62]:
simpbrand

,brand,similar,same
0,neo-tec妮傲絲翠,"[NEO-TEC妮傲絲翠, NEO-TEC 妮傲絲翠]",
1,philips飛利浦,"[飛利浦, 飛利浦 Philips, 飛利浦 PHILIPS]",
2,senka專科,"[專科, SENKA 專科]",
3,fancl芳珂,"[fancl, FANCL 芳珂, FANCL]",
4,red bull紅牛,[紅牛 Red Bull],
...,...,...,...
6458,plantur 21,[],plantur
6459,plantur 39,[],plantur
6460,Plantur 39,[],plantur
6461,Plantur 21,[],plantur


In [63]:
data['final_brand'] = ''
for i in tqdm(range(len(data))):
    srwd = data['cleaned_brand'][i]
    searchres = simpbrand[simpbrand['brand'] == srwd]

    if len(searchres) == 0:
        data['final_brand'][i] = srwd
        print("index:", i)
        print("No Matched Search:", srwd)
    elif len(searchres) == 1:
        samewd = simpbrand['same'][searchres.index[0]] 
        if samewd == '': # 是主值
            data['final_brand'][i] = srwd
        else: # 不是主值
            data['final_brand'][i] = samewd
    else:
        data['final_brand'][i] = srwd
        print("index:", i)
        print("Multiple Results:", srwd)
        
print('-----END-----')

100%|███████████████████████████████████| 70046/70046 [00:35<00:00, 1956.09it/s]

-----END-----


In [64]:
data

,name,brand,clean_name,clean_brand,brand_ch,brand_en,cleaned_brand,final_brand
0,元山熱水瓶ys-540ap,元山,元山熱水瓶ys540ap,元山,元山,,元山,元山
1,356612@姍拉娜治痘洗面乳150g,姍拉娜,姍拉娜治痘洗面乳,姍拉娜,姍拉娜,,姍拉娜,姍拉娜
2,台鹽海洋鹼性離子水 600ml,台鹽,台鹽海洋鹼性離子水,台鹽,台鹽,,台鹽,台鹽
3,妮傲絲翠果酸深層保養乳液,NEO-TEC妮傲絲翠,妮傲絲翠果酸深層保養乳液,neo-tec妮傲絲翠,妮傲絲翠,neo-tec,neo-tec妮傲絲翠,neo-tec妮傲絲翠
4,舒潔棉柔舒適迪士尼抽取式衛生紙 100抽16入,舒潔,舒潔棉柔舒適迪士尼抽取式衛生紙,舒潔,舒潔,,舒潔,舒潔
...,...,...,...,...,...,...,...,...
70041,惠而浦16公斤瓦斯型滾筒蒸氣(可堆疊)乾衣機8twgd6622hw,惠而浦 Whirlpool,惠而浦瓦斯型滾筒蒸氣可堆疊乾衣機8twgd6622hw,惠而浦 whirlpool,惠而浦,whirlpool,whirlpool惠而浦,whirlpool惠而浦
70042,康乃馨成人紙尿褲l,康乃馨,康乃馨成人紙尿褲l,康乃馨,康乃馨,,康乃馨,康乃馨
70043,日本芮芙茹零矽靈洗髮露460ml頭皮保養,芮芙茹 Reveur,日本芮芙茹零矽靈洗髮露頭皮保養,芮芙茹 reveur,芮芙茹,reveur,reveur芮芙茹,reveur芮芙茹
70044,crest長效鎖白牙膏-輕柔鑽白99g,crest,crest長效鎖白牙膏輕柔鑽白,crest,,crest,crest,crest


In [65]:
data = data[['clean_name', 'final_brand']]
data.to_csv('cleaned_brand_train.csv')

## 整理Test資料集

In [4]:
testdata = pd.read_csv('brand_test(question).csv')
testdata = testdata[['name','brand']]
testdata['brand'] = ''

In [5]:
testdata

,name,brand
0,【heme 喜蜜】輕透柔光粉底液 SPF50+ ★★★ - 30ml(2入組),
1,【肌研】卵肌溫和去角質化粧水170ml 2入,
2,"C.M康媚絲洗髮精,強化",
3,BR超柔潤保濕面膜 5片,
4,(即期品)(買1送1)dejavu 睫采分明立體睫毛膏_黑色3.1g(效期2022/6/30),
...,...,...
97154,石澤研究所毛穴撫子角質對策泡洗顏100g,
97155,金牌台灣啤酒330ml6入,
97156,日清 中杯麵 醬油,
97157,杜蕾斯活力裝衛生套３個／盒,


In [6]:
def cleanf(text):
    
    if not text.isnumeric():
        # 中文量詞
        remove_measure = re.sub(r'''(x|\*)?([0-9]+( +)?(入|支|包|片|瓶|罐|盒|杯|(顆)?袋)+裝|[0-9]+\.[0-9]+包|[0-9]+\.[0-9]+片|[0-9]+\.[0-9]+瓶|[0-9]+\.[0-9]+號|[0-9]+\.[0-9]+罐|[0-9]+\.[0-9]+盒|[0-9]+\.[0-9]+組|[0-9]+\.[0-9]+杯|[0-9]+\.[0-9]+公分|[0-9]+\.[0-9]+公克|[0-9]+\.[0-9]+公斤|[0-9]+\.[0-9]+公升|[0-9]+\.[0-9]+升|[0-9]+\.[0-9]+人份|[0-9]+\.[0-9]+支|[0-9]+\.[0-9]+元|[0-9]+\.[0-9]+支入|[0-9]+\.[0-9]+毫升|[0-9]+\.[0-9]+毫克|[0-9]+\.[0-9]+件|[0-9]+\.[0-9]+日|[0-9]+\.[0-9]+克|[0-9]+\.[0-9]+袋|[0-9]+\.[0-9]+斤|[0-9]+\.[0-9]+磅|[0-9]+\.[0-9]+箱|[0-9]+\.[0-9]+裝|[0-9]+\.[0-9]+顆|[0-9]+\.[0-9]+粒|[0-9]+\.[0-9]+錠|[0-9]+\.[0-9]+桶|[0-9]+\.[0-9]+塊|[0-9]+\.[0-9]+束|[0-9]+\.[0-9]+條|[0-9]+\.[0-9]+吋|[0-9]+\.[0-9]+抽|[0-9]+\.[0-9]+個|[0-9]+\.[0-9]+枚|[0-9]+\.[0-9]+張|[0-9]+\.[0-9]+串|[0-9]+\.[0-9]+捲|[0-9]+\.[0-9]+雙|[0-9]+\.[0-9]+加侖|[0-9]+\.[0-9]+公尺|[0-9]+\.[0-9]+頁|[0-9]+\.[0-9]+歲|[0-9]+\.[0-9]+折|[0-9]+\.[0-9]+款|[0-9]+\.[0-9]+本|[0-9]+\.[0-9]+筒|[0-9]+\.[0-9]+入|下殺[0-9]+\.[0-9]|[0-9]+支入|[0-9]+公克|[0-9]+包|[0-9]+片|[0-9]+瓶|[0-9]+號|[0-9]+罐|[0-9]+盒|[0-9]+組|[0-9]+公分|[0-9]+( )?公斤|[0-9]+入(組|盒)|[0-9]+公升|[0-9]+( )?份入|[0-9]+毫升|[0-9]+毫克|[0-9]+克|[0-9]+袋|[0-9]+( )?磅|[0-9]+箱|[0-9]+裝|[0-9]+顆((袋|盒)裝)?|[0-9]+粒|[0-9]+錠|[0-9]+桶|[0-9]+塊|[0-9]+束|[0-9]+條|[0-9]+吋|[0-9]+抽|[0-9]+個|[0-9]+枚(入)?|[0-9]+張|[0-9]+串|[0-9]+捲|[0-9]+雙|[0-9]+加侖|[0-9]+公尺|[0-9]+頁|[0-9]+本|[0-9]+筒|[0-9]+(\+[0-9])?( +)?入|[0-9]+件(組)?|[0-9]+日|[0-9]+升|[0-9]+人份|[0-9]+碗(入)?|[0-9]+杯|[0-9]+(-[0-9]+)?歲|[0-9]+折|[0-9]+款|[0-9]+支|[0-9]+元|[0-9]+( )?吋|[0-9]+斤|[0-9]+坪|下殺[0-9])''',"",str(text))

        # 英文量詞
        remove_egmeasure = re.sub(r'''[0-9]+\.[0-9]+ml|[0-9]+\.[0-9]+l|[0-9]+\.[0-9]+oz|[0-9]+\.[0-9]+%|[0-9]+\.[0-9]+g|[0-9]+\.[0-9]+kg|[0-9]+\.[0-9]+cm|[0-9]+\.[0-9]+mm|[0-9]+\.[0-9]+mg|[0-9]+\.[0-9]+m|[0-9]+\.[0-9]+cc|[0-9]+\.[0-9]+c.c.|[0-9]+\.[0-9]+CC|[0-9]+\.[0-9]+gm|[0-9]+\.[0-9]+pg|$[0-9]+\.[0-9]+|[0-9]+ml|[0-9]+ ml|[0-9]+l|[0-9]+oz|[0-9]+g|[0-9]+ g|[0-9]+kg|([0-9]+\*)?[0-9]+cm|[0-9]+mm|[0-9]+mg|[0-9]+m|[0-9]+cc|[0-9]+c.c.|[0-9]+gm|[0-9]+pg|[0-9]+%|$[0-9]''', "", remove_measure)

        # 複合詞
        remove_both = re.sub(r'''^[0-9]{6,15}|[0-9]+@|([0-9]+)?/+(包|裝|盒|kg|組|支|瓶|袋|罐|箱|顆)|\d{4}/\d{1,2}/\d{1,2}|\d{4}/\d{1,2}|\d{4}.\d{1,2}.\d{1,2}|\d{4}.\d{1,2}|長[0-9]+(\.[0-9]+)?x寬[0-9]+(\.[0-9]+)?x高([0-9]+\.[0-9]+)?''', "", remove_egmeasure)

        # 行銷詞
        remove_buss = re.sub(r'''買一送一|買+[0-9]+送+[0-9]|[0-9]+送+[0-9]|現貨|蝦皮24h|24h|24 h|代收代付\-|代收代付|任選|特惠組|(保存|最低)(期限|效期至|效期至)|效期\d{4}|效期|期限至|期限|效期至|下單|出貨|官方直營|免運(費)?|型號(:)?\w+''', "", remove_both)

        # 特殊符號與自訂
        remove_symbol = re.sub(r'''\d\+\d|[0-9]+\.[0-9]+$|[0-9]+\$|\d{1,3}$|^\s+|\s+$|\*[0-9]+[a-z]+|\~[a-z]+\~|^[a-z]\-|\![a-z+]\-|(x|\*)[0-9]|(x|\*)+[0-9]+\.[0-9]|[!;:"'{}'.,/?\@#<>&※〔〕【】「」『』\[\]《》〈〉«»、∥=→↑。\│±()+\-^~_\*\$|♦▼▶★☆❤㊣╳╱♠⚡️◆✱▲↘✬✪™∮☛●◤◢≡⭐️✴️✨☀◖◗®]''', "", remove_buss) 
    else:
        remove_symbol = text
    return remove_symbol

In [7]:
# 大寫轉小寫
testdata['name'] = testdata['name'].str.lower()

for i in tqdm(range(len(testdata))):
    try:
        testdata['name'][i] = strQ2B(testdata['name'][i])
    except:
        pass

100%|████████████████████████████████| 97159/97159 [00:00<00:00, 1985386.04it/s]


In [9]:
testdata['clean_name'] = ''
testdata['clean_name'] = testdata.name.apply(cleanf)

In [10]:
testdata = testdata[['name', 'clean_name','brand']]
testdata.to_csv('cleaned_brand_test.csv')